<a href="https://colab.research.google.com/github/alessandronascimento/pyLiBELa/blob/main/Colabs/pyLiBELa_SB2021_cuda_global.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title Downloading dependencies {display-mode: "form"}

%%capture
! apt-get install python-dev-is-python3 zlib1g-dev libeigen3-dev libgsl-dev libnlopt-cxx-dev
!pip install ipython-autotime
%load_ext autotime
!pip3 install condacolab
import condacolab
condacolab.install()
!mamba install openbabel
import multiprocessing
from joblib import Parallel, delayed

time: 51.3 s (started: 2023-10-25 22:58:09 +00:00)


In [1]:
#@title Installing pyLiBELa {display-mode: "form"}

%%capture
! rm -f Makefile*
! git clone --branch main https://github.com/alessandronascimento/pyLiBELa.git
! wget https://raw.githubusercontent.com/alessandronascimento/pyLiBELa/main/Colabs/Makefile
! mv pyLiBELa/src src
! mv pyLiBELa/test test
! rm -rf pyLiBELa
! mkdir -p obj
! sed -i 's+-I/usr/include/openbabel3+-I/usr/local/include/openbabel3+g' Makefile
! make -j2

Cloning into 'pyLiBELa'...
remote: Enumerating objects: 2965, done.
remote: Counting objects: 100% (437/437), done.
remote: Compressing objects: 100% (202/202), done.
remote: Total 2965 (delta 275), reused 366 (delta 229), pack-reused 2528
Receiving objects: 100% (2965/2965), 40.32 MiB | 18.95 MiB/s, done.
Resolving deltas: 100% (1991/1991), done.
--2023-10-25 22:59:11--  https://raw.githubusercontent.com/alessandronascimento/pyLiBELa/main/Colabs/Makefile
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6093 (6.0K) [text/plain]
Saving to: ‘Makefile’

Makefile            100%[===================>]   5.95K  --.-KB/s    in 0s      

2023-10-25 22:59:11 (56.1 MB/s) - ‘Makefile’ saved [6093/6093]

g++ -fPIC -O3 -I/usr/include/python3.10 -I/usr/local/include/openba

In [3]:
#@title Importing pyLiBELa {display-mode: "form"}
import random

try:
  from pyPARSER import *
  from pyMol2 import *
  from pyWRITER import *
  from pyGrid import *
  from pyCOORD_MC import *
  from pyFindHB import *
  from pyEnergy2 import *
  from pyGaussian import *
  from pyConformer import *
  from pyRAND import *
  from pyMcEntropy import *
  from pySA import *
  from pyOptimizer import *
  from pyMC import *
  from pyFullSearch import *
  from pyDocker import *
  print('pyLiBELa is imported!')
except ImportError:
  print('An ImportError occurred, try running this cell or the one above it again!')

pyLiBELa is imported!


In [8]:
#@title Getting SB2021 data {display-mode: "form"}

%%capture
from google.colab import drive
drive.mount('/content/drive/')
sb_folder = '/content/drive/MyDrive/pyLiBELa/SB/' #@param {type:"string"}

%cd $sb_folder
targets=[]

targets_file = open('list', 'r');
for line in targets_file:
  targets.append(line.strip())
targets_file.close()
#targets = targets[:-2] #os últimos 2 são list e list.txt, só quero os pdb

time: 2.34 s (started: 2023-10-25 23:36:57 +00:00)


In [18]:
#@title Docking parameters {display-mode: "form"}

import os
import timeit
import numpy as np
Input = PARSER()

Input.generate_conformers = True;
Input.dock_parallel = False;
Input.parallel_jobs = 1;
Input.write_grids = False;
Input.load_grid_from_file = True;
Input.use_grids = True
Input.write_mol2 = True
Input.atom_limit = 10000 #@param {type:"number"}

scoring_function = "0" #@param ["0", "1", "2", "3"]
Input.dielectric_model = "r" #@param ["r", "constant"]
#Input.diel = 2.0;
Input.scoring_function = int(scoring_function)
grid_dimension = 30.0 #@param {type:"number"}
Input.grid_prefix = 'McGrid'
Input.grid_spacing = 0.4 #@param {type:"number"}
Input.solvation_alpha = 0.1 #@param {type:"number"}
Input.solvation_beta = -0.005 #@param {type:"number"}

# Optimization parameter:
Input.min_tol = 1E-4;
Input.min_delta = 1E-4;
Input.dock_min_tol = 1E-4;
search_box = 8.0 #@param {type:"number"}
Input.timeout = 20 #@param {type:"number"}
Input.min_timeout = 30 #@param {type:"number"}
Input.overlay_optimizer = "mma" #@param ["mma", "ln_auglag", "subplex", "none"]
Input.energy_optimizer = "stogo" #@param ["direct", "isres", "crs", "esch", "stogo", "mma", "simplex", "none"]
if (Input.scoring_function < 3):
    delta = 2.5 #@param {type:"number"}
    Input.deltaij6 = (delta*delta*delta*delta*delta*delta)
    delta_es = 2.5 #@param {type:"number"}
    Input.deltaij_es6 = pow(delta_es, 6);
    Input.deltaij_es3 = (delta_es*delta_es*delta_es)

Input.conf_search_trials = 10000;
Input.conformers_to_evaluate = 2;
Input.lig_conformers = 10;

Input.search_box_x, Input.search_box_y, Input.search_box_z = search_box, search_box, search_box;
Input.x_dim, Input.y_dim, Input.z_dim = grid_dimension, grid_dimension, grid_dimension;

time: 2.25 ms (started: 2023-10-25 23:43:30 +00:00)


In [19]:
#@title Docking {display-mode: "form"}

%reload_ext autotime
import os


print('#%10s %10s' % ('TARGET', 'RMSD(Ang)'))

targets_random=random.sample(targets, len(targets));

target_cut = targets[1166:]

k=0


for target in target_cut:
  print(k)
  k+=1

  folder = sb_folder+target


  %cd $folder
  mma_folder = "self_docking_mma_third"
 # ! mkdir -p $mma_folder
  %cd $mma_folder

  subfolder = "self_docking_"+Input.energy_optimizer+'_second'
 # !rm -rf McLiBELa_dock*

  if subfolder not in os.listdir('..'):
    %cd ..
    !mkdir -p $subfolder
    %cd $mma_folder

  if 'McLiBELa_dock.mol2.gz' not in os.listdir("../"+subfolder):
    if "McGrid.grid" in os.listdir():

      %cd ..


      !mkdir -p $subfolder
      !rm -rf $subfolder/*

      !cp self_docking_mma/McGrid.grid $subfolder/McGrid.grid
      !cp self_docking_mma/lig2.smi $subfolder/lig2.smi


      %cd $subfolder

      print()
      print()

      ! cp ../*lig.am1bcc.mol2.gz lig.mol2.gz
      ! cp ../*.rec.clean.mol2.gz rec.mol2.gz


      #insmiles = open('lig2.smi', 'r');
      #line=insmiles.readline().strip()
      #line2 = line.split()
      #ligand_smiles = line2[0]
      #insmiles.close();

      lig_src =  'lig.mol2.gz'
      rec_src = 'rec.mol2.gz'

      try:

        REC = Mol2(Input, rec_src)
        RefLig = Mol2(Input, lig_src)
        Lig2 = Mol2(Input, lig_src)

        ligcheck = True

        #ligcheck = Lig2.parse_smiles(Input, ligand_smiles, 'LIG')

      except (RuntimeError, MemoryError):
        print("Skipping, something went wrong...Check the logs.")
        log = open("log.txt", 'w')
        log.write("Something went wrong, this target was skipped.")
        log.close()
        continue


      if (ligcheck):

       Writer = WRITER(Input)
       Coord = COORD_MC()
       HB = FindHB()
       Ene  = Energy2(Input)

       for i in range(len(REC.residue_pointer)-1):
         HB.parse_residue(REC.residue_pointer[i]-1, REC.residue_pointer[i+1]-2, REC.resnames[i], REC, RefLig, 9.0);
         HB.find_ligandHB(lig_src, RefLig);

       Conf = Conformer();
       Conf.generate_conformers_confab(Input, Lig2, lig_src);

       Dock = Docker(Writer)
       center = Coord.compute_com(RefLig)

       start_time = timeit.default_timer()
       Grids = Grid(Input, Writer)
       Grids.load_grids_from_file()
       time_grid = timeit.default_timer() - start_time
       print(time_grid)


       start_energy = Ene.compute_ene(REC, RefLig, RefLig.xyz);

       grid_energy = Ene.compute_ene(Grids, RefLig, RefLig.xyz);

       start_time = timeit.default_timer()
       Dock.run(REC, Lig2, RefLig, center, Input, Grids, 0)
       time_dock = timeit.default_timer() - start_time


       log = open("log.txt", 'a')

       header = "target\tREC.N\tRefLig.N\tLig2.N\ttime_grid\tstart_energy\tgrid_energy\ttime_dock\n"
       log.write(header)

       data = "%s\t%d\t%d\t%d\t%.3f\t%.3f\t%.3f\t%.3f\n" %(target,REC.N, RefLig.N, Lig2.N,time_grid,start_energy,grid_energy, time_dock)
       log.write(data)
       print(data)
       log.close()

  else:
    print('Target already docked!')





#    TARGET  RMSD(Ang)
0
/content/drive/MyDrive/pyLiBELa/SB/9AAT
/content/drive/MyDrive/pyLiBELa/SB/9AAT/self_docking_mma_third
Target already docked!
1
/content/drive/MyDrive/pyLiBELa/SB/9HVP
/content/drive/MyDrive/pyLiBELa/SB/9HVP/self_docking_mma_third
/content/drive/MyDrive/pyLiBELa/SB/9HVP
/content/drive/MyDrive/pyLiBELa/SB/9HVP/self_docking_mma_third
/content/drive/MyDrive/pyLiBELa/SB/9HVP
/content/drive/MyDrive/pyLiBELa/SB/9HVP/self_docking_stogo_second


0.09187210699974457
9HVP	3134	106	106	0.092	-64.704	-63.175	31.020

2
/content/drive/MyDrive/pyLiBELa/SB/9ICD
/content/drive/MyDrive/pyLiBELa/SB/9ICD/self_docking_mma_third
/content/drive/MyDrive/pyLiBELa/SB/9ICD
/content/drive/MyDrive/pyLiBELa/SB/9ICD/self_docking_mma_third
/content/drive/MyDrive/pyLiBELa/SB/9ICD
/content/drive/MyDrive/pyLiBELa/SB/9ICD/self_docking_stogo_second


0.13340949599978558
9ICD	6421	38	38	0.133	-42.670	-42.778	62.640

time: 1min 51s (started: 2023-10-25 23:43:32 +00:00)
